In [8]:
from numpy import exp, array, random, dot
import joblib
import time

In [28]:
%%time

_period = 'm5'
class NeuronLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1


class NeuralNetwork():
    def __init__(self, layer1, layer2, layer3):
        self.layer1 = layer1
        self.layer2 = layer2

    # The Sigmoid function, which describes an S shaped curve.
    # We pass the weighted sum of the inputs through this function to
    # normalise them between 0 and 1.
    def __sigmoid(self, x):
        return 1 / (1 + exp(-x))

    # The derivative of the Sigmoid function.
    # This is the gradient of the Sigmoid curve.
    # It indicates how confident we are about the existing weight.
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    # We train the neural network through a process of trial and error.
    # Adjusting the synaptic weights each time.
    def train(self, training_set_inputs, training_set_outputs, number_of_training_iterations):
        for iteration in range(number_of_training_iterations):
            # Pass the training set through our neural network
            output_from_layer_1, output_from_layer_2 = self.think(training_set_inputs)

            # Calculate the error for layer 2 (The difference between the desired output
            # and the predicted output).
            layer2_error = training_set_outputs - output_from_layer_2
            layer2_delta = layer2_error * self.__sigmoid_derivative(output_from_layer_2)

            # Calculate the error for layer 1 (By looking at the weights in layer 1,
            # we can determine by how much layer 1 contributed to the error in layer 2).
            layer1_error = layer2_delta.dot(self.layer2.synaptic_weights.T)
            layer1_delta = layer1_error * self.__sigmoid_derivative(output_from_layer_1)

            # Calculate how much to adjust the weights by
            layer1_adjustment = training_set_inputs.T.dot(layer1_delta)
            layer2_adjustment = output_from_layer_1.T.dot(layer2_delta)

            # Adjust the weights.
            self.layer1.synaptic_weights += layer1_adjustment
            self.layer2.synaptic_weights += layer2_adjustment

    # The neural network thinks.
    def think(self, inputs):
        output_from_layer1 = self.__sigmoid(dot(inputs, self.layer1.synaptic_weights))
        output_from_layer2 = self.__sigmoid(dot(output_from_layer1, self.layer2.synaptic_weights))
        return output_from_layer1, output_from_layer2

    # The neural network prints its weights
    def print_weights(self):
        print( "    Layer 1 (4 neurons, each with 3 inputs): ")
        print (self.layer1.synaptic_weights)
        print ("    Layer 2 (1 neuron, with 4 inputs):")
        print (self.layer2.synaptic_weights)


#Seed the random number generator
random.seed(42)

# Create layer 1 (52 neurons, each with 52 inputs)
layer1 = NeuronLayer(104, 52)

# Create layer 2 (a single neuron with 4 inputs)
layer2 = NeuronLayer(52, 26)

# Create layer 2 (a single neuron with 4 inputs)
layer3 = NeuronLayer(26, 1)

# Combine the layers to create a neural network
neural_network = NeuralNetwork(layer1, layer2, layer3)

print ("Stage 1) Random starting synaptic weights: ")
neural_network.print_weights()

# The training set. We have 7 examples, each consisting of 3 input values
# and 1 output value.

features = joblib.load('FEATURES/featuring_'+_period+'.dag')
features = features.dropna()
training_set_inputs = array(features.drop(['Date','Symbol','TRACKER','Signal'],axis=1).values.reshape(714845,52)) # array([[0, 0, 1], [0, 1, 1], [1, 0, 1], [0, 1, 0], [1, 0, 0], [1, 1, 1], [0, 0, 0]])
training_set_outputs = array(features.Signal).T.reshape(-1,1) # array([[0, 1, 1, 1, 1, 0, 0]]).T

# Train the neural network using the training set.
# Do it 60,000 times and make small adjustments each time.
neural_network.train(training_set_inputs, training_set_outputs, 60000)

print ("Stage 2) New synaptic weights after training: ")
neural_network.print_weights()

# Test the neural network with a new situation.
print( "Stage 3) Considering a new situation [1, 1, 0] -> ?: ")
hidden_state, output = neural_network.think(array([1, 1, 1, 0]))
print (output)

Stage 1) Random starting synaptic weights: 
    Layer 1 (4 neurons, each with 3 inputs): 
[[-0.25091976  0.90142861  0.46398788 ...  0.27282082 -0.37128804
   0.01714138]
 [ 0.81513295 -0.50141554 -0.17923415 ... -0.9816059  -0.79705691
   0.32700354]
 [-0.98987683 -0.6783839   0.09746758 ... -0.4583355  -0.12205716
  -0.84308724]
 ...
 [ 0.68798694 -0.47518325 -0.15657476 ... -0.87241089 -0.22555372
  -0.43446909]
 [-0.39101915  0.96589376  0.28555483 ... -0.97897355  0.52377979
   0.78660899]
 [-0.13432016  0.07108495 -0.04603321 ...  0.79111148  0.93324444
  -0.11312179]]
    Layer 2 (1 neuron, with 4 inputs):
[[-0.35800591  0.05218803  0.37553841 ... -0.21816515  0.95291306
   0.90933271]
 [ 0.60027517 -0.51668216 -0.86392537 ... -0.07202917 -0.73343148
   0.85549147]
 [ 0.7354042   0.41524117 -0.78757234 ...  0.99534453 -0.47289704
  -0.44173906]
 ...
 [ 0.0465725  -0.76650419 -0.74186975 ... -0.64333698 -0.52126033
  -0.16406298]
 [-0.49761599 -0.35886483 -0.73004303 ...  0.63592

ValueError: shapes (714845,104) and (26,52) not aligned: 104 (dim 1) != 26 (dim 0)

In [20]:
training_set_inputs

array([[ 1.43164   ,  1.43164   ,  1.43037   , ...,         nan,
                nan,         nan],
       [ 1.43109   ,  1.43113   ,  1.4296    , ...,         nan,
                nan,         nan],
       [ 1.43071   ,  1.43084   ,  1.42905   , ...,         nan,
                nan,         nan],
       ...,
       [ 1.212055  ,  1.212135  ,  1.21182   , ..., -0.10474002,
        21.21185188, 47.51342068],
       [ 1.21202   ,  1.21215   ,  1.21161   , ..., -0.33062815,
        21.21172688, 47.51329568],
       [ 1.211895  ,  1.212145  ,  1.21177   , ...,  0.37000027,
        21.21187688, 47.51344568]])